In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, LSTM, GRU
from sklearn.utils import resample
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
# Load the dataset
df = pd.read_csv('statlog.csv', header = None)

df.columns = ['age', 'sex', 'cp', 'trestbps', 'chol',
              'fbs', 'restecg', 'thalach', 'exang', 
              'oldpeak', 'slope', 'ca', 'thal', 'target']

df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,2
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,1
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,2
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,1
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,52,1,3,172,199,1,0,162,0,0.5,1,0,7,1
266,44,1,2,120,263,0,0,173,0,0.0,1,0,7,1
267,56,0,2,140,294,0,2,153,0,1.3,2,0,3,1
268,57,1,4,140,192,0,0,148,0,0.4,2,0,6,1


In [3]:
### 1 = male, 0 = female
df.isnull().sum()

df['target'] = df.target.map({0: 0, 1: 0, 2: 1, 3: 1, 4: 1})
df['sex'] = df.sex.map({0: 'female', 1: 'male'})
df['thal'] = df.thal.fillna(df.thal.mean())
df['ca'] = df.ca.fillna(df.ca.mean())
df['sex'] = df.sex.map({'female': 0, 'male': 1})

In [4]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,1
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,0
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,1
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,0
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,52,1,3,172,199,1,0,162,0,0.5,1,0,7,0
266,44,1,2,120,263,0,0,173,0,0.0,1,0,7,0
267,56,0,2,140,294,0,2,153,0,1.3,2,0,3,0
268,57,1,4,140,192,0,0,148,0,0.4,2,0,6,0


In [5]:
# Separate features and target variable
X = df.drop('target', axis=1)
y = df['target']


In [7]:

# 1. Load and preprocess the data
# Assuming you have your data in the X and y variables

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [8]:
input_shape = (X_train.shape[1], 1)
input_shape_cnn = (X_train.shape[1], 1)

In [9]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [10]:
def build_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=11, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [11]:
# Train the individual models
cnn_model = build_cnn_model(input_shape)
cnn_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
             callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
4/4 [==============================] - 2s 201ms/step - loss: 0.6615 - accuracy: 0.6561 - val_loss: 0.6549 - val_accuracy: 0.6914
Epoch 2/100
4/4 [==============================] - 0s 40ms/step - loss: 0.4911 - accuracy: 0.7672 - val_loss: 0.6362 - val_accuracy: 0.7037
Epoch 3/100
4/4 [==============================] - 0s 46ms/step - loss: 0.5554 - accuracy: 0.7672 - val_loss: 0.6260 - val_accuracy: 0.7037
Epoch 4/100
4/4 [==============================] - 0s 44ms/step - loss: 0.4420 - accuracy: 0.7937 - val_loss: 0.6219 - val_accuracy: 0.7037
Epoch 5/100
4/4 [==============================] - 0s 44ms/step - loss: 0.4802 - accuracy: 0.7884 - val_loss: 0.6205 - val_accuracy: 0.7037
Epoch 6/100
4/4 [==============================] - 0s 41ms/step - loss: 0.4614 - accuracy: 0.7989 - val_loss: 0.6181 - val_accuracy: 0.7037
Epoch 7/100
4/4 [==============================] - 0s 43ms/step - loss: 0.4653 - accuracy: 0.8042 - val_loss: 0.6180 - val_accuracy: 0.7037
Epoch 8/100
4/4 [==

4/4 [==============================] - 0s 41ms/step - loss: 0.1008 - accuracy: 0.9577 - val_loss: 0.4753 - val_accuracy: 0.7654
Epoch 60/100
4/4 [==============================] - 0s 27ms/step - loss: 0.1400 - accuracy: 0.9365 - val_loss: 0.4798 - val_accuracy: 0.7901
Epoch 61/100
4/4 [==============================] - 0s 23ms/step - loss: 0.1125 - accuracy: 0.9577 - val_loss: 0.4822 - val_accuracy: 0.7531
Epoch 62/100
4/4 [==============================] - 0s 23ms/step - loss: 0.1049 - accuracy: 0.9577 - val_loss: 0.4824 - val_accuracy: 0.7654
Epoch 63/100
4/4 [==============================] - 0s 20ms/step - loss: 0.1066 - accuracy: 0.9630 - val_loss: 0.4832 - val_accuracy: 0.7778
Epoch 64/100
4/4 [==============================] - 0s 20ms/step - loss: 0.0869 - accuracy: 0.9735 - val_loss: 0.4792 - val_accuracy: 0.8025
Epoch 65/100
4/4 [==============================] - 0s 47ms/step - loss: 0.0642 - accuracy: 0.9788 - val_loss: 0.4732 - val_accuracy: 0.7778
Epoch 66/100
4/4 [========

In [12]:
# 8. Evaluate the model
test_loss, test_accuracy = cnn_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


3/3 [==============================] - 0s 4ms/step - loss: 0.4474 - accuracy: 0.7901
Test Loss: 0.4474
Test Accuracy: 0.7901


In [13]:
# Make predictions
y_pred = cnn_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


3/3 [==============================] - 0s 5ms/step
Confusion Matrix:
[[44  5]
 [12 20]]

Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.90      0.84        49
           1       0.80      0.62      0.70        32

    accuracy                           0.79        81
   macro avg       0.79      0.76      0.77        81
weighted avg       0.79      0.79      0.78        81



In [14]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [15]:
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(11, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(32, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(LSTM(64, return_sequences=True))
    model.add(LSTM(32))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [16]:
lstm_model = build_lstm_model(input_shape)
lstm_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
              callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
4/4 [==============================] - 13s 883ms/step - loss: 0.6819 - accuracy: 0.5344 - val_loss: 0.6862 - val_accuracy: 0.6049
Epoch 2/100
4/4 [==============================] - 0s 88ms/step - loss: 0.6364 - accuracy: 0.7407 - val_loss: 0.6720 - val_accuracy: 0.6049
Epoch 3/100
4/4 [==============================] - 0s 87ms/step - loss: 0.5958 - accuracy: 0.7354 - val_loss: 0.6512 - val_accuracy: 0.6049
Epoch 4/100
4/4 [==============================] - 0s 87ms/step - loss: 0.5756 - accuracy: 0.7354 - val_loss: 0.6350 - val_accuracy: 0.6296
Epoch 5/100
4/4 [==============================] - 0s 87ms/step - loss: 0.5727 - accuracy: 0.7513 - val_loss: 0.6322 - val_accuracy: 0.6667
Epoch 6/100
4/4 [==============================] - 0s 51ms/step - loss: 0.5588 - accuracy: 0.7407 - val_loss: 0.6349 - val_accuracy: 0.6667
Epoch 7/100
4/4 [==============================] - 0s 53ms/step - loss: 0.5456 - accuracy: 0.7249 - val_loss: 0.6412 - val_accuracy: 0.6914
Epoch 8/100
4/4 [=

In [17]:
# 8. Evaluate the model
test_loss, test_accuracy = lstm_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


3/3 [==============================] - 0s 11ms/step - loss: 0.6322 - accuracy: 0.6667
Test Loss: 0.6322
Test Accuracy: 0.6667


In [18]:
# Make predictions
y_pred = lstm_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


3/3 [==============================] - 2s 10ms/step
Confusion Matrix:
[[49  0]
 [27  5]]

Classification Report:
              precision    recall  f1-score   support

           0       0.64      1.00      0.78        49
           1       1.00      0.16      0.27        32

    accuracy                           0.67        81
   macro avg       0.82      0.58      0.53        81
weighted avg       0.79      0.67      0.58        81



In [19]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [20]:
def build_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(11, return_sequences=True, input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(32, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(64, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(128, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(128))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [21]:
gru_model = build_gru_model(input_shape)
gru_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
             callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
4/4 [==============================] - 21s 1s/step - loss: 0.6154 - accuracy: 0.6667 - val_loss: 0.6871 - val_accuracy: 0.6420
Epoch 2/100
4/4 [==============================] - 0s 138ms/step - loss: 0.5101 - accuracy: 0.7884 - val_loss: 0.6817 - val_accuracy: 0.6173
Epoch 3/100
4/4 [==============================] - 1s 144ms/step - loss: 0.4851 - accuracy: 0.8042 - val_loss: 0.6790 - val_accuracy: 0.6049
Epoch 4/100
4/4 [==============================] - 1s 141ms/step - loss: 0.4167 - accuracy: 0.8360 - val_loss: 0.6761 - val_accuracy: 0.6049
Epoch 5/100
4/4 [==============================] - 1s 144ms/step - loss: 0.3488 - accuracy: 0.8677 - val_loss: 0.6740 - val_accuracy: 0.6049
Epoch 6/100
4/4 [==============================] - 1s 144ms/step - loss: 0.3863 - accuracy: 0.8307 - val_loss: 0.6712 - val_accuracy: 0.6173
Epoch 7/100
4/4 [==============================] - 1s 148ms/step - loss: 0.4442 - accuracy: 0.8148 - val_loss: 0.6689 - val_accuracy: 0.6543
Epoch 8/100
4/4

In [22]:
# 8. Evaluate the model
test_loss, test_accuracy = gru_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


3/3 [==============================] - 0s 20ms/step - loss: 0.6662 - accuracy: 0.6914
Test Loss: 0.6662
Test Accuracy: 0.6914


In [23]:
# Make predictions
y_pred = gru_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


3/3 [==============================] - 3s 16ms/step
Confusion Matrix:
[[49  0]
 [25  7]]

Classification Report:
              precision    recall  f1-score   support

           0       0.66      1.00      0.80        49
           1       1.00      0.22      0.36        32

    accuracy                           0.69        81
   macro avg       0.83      0.61      0.58        81
weighted avg       0.80      0.69      0.62        81



In [24]:
# Make predictions using individual models
cnn_predictions = (cnn_model.predict(X_test) > 0.5).astype("int32")
lstm_predictions = (lstm_model.predict(X_test) > 0.5).astype("int32")
gru_predictions = (gru_model.predict(X_test) > 0.5).astype("int32")



3/3 [==============================] - 0s 15ms/step


In [25]:
# Combine predictions using a majority voting ensemble
ensemble_predictions = np.mean([cnn_predictions, lstm_predictions, gru_predictions], axis=0) >= 0.5
ensemble_predictions = ensemble_predictions.astype("int32")

In [26]:
# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.7037


In [27]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, ensemble_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, ensemble_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")


Confusion Matrix:
[[49  0]
 [24  8]]
Classification Report:
              precision    recall  f1-score   support

           0       0.67      1.00      0.80        49
           1       1.00      0.25      0.40        32

    accuracy                           0.70        81
   macro avg       0.84      0.62      0.60        81
weighted avg       0.80      0.70      0.64        81

Ensemble Accuracy: 0.7037


In [28]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.utils import resample
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.model_selection import cross_val_score

In [29]:
# List of base models

lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

svm_l_model = SVC(kernel="linear")
svm_l_model.fit(X_train, y_train)

svm_p_model = SVC(kernel="poly")
svm_p_model.fit(X_train, y_train)

svm_r_model = SVC(kernel="rbf")
svm_r_model.fit(X_train, y_train)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)

gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

ab_model = AdaBoostClassifier(n_estimators=100, random_state=42)
ab_model.fit(X_train, y_train)

xgb_model = XGBClassifier(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

gpc_model = GaussianProcessClassifier(random_state=42)
gpc_model.fit(X_train, y_train)

GaussianProcessClassifier(random_state=42)

In [30]:
lr_predictions = lr_model.predict(X_test)
svm_l_predictions = svm_l_model.predict(X_test)
svm_p_predictions = svm_p_model.predict(X_test)
svm_r_predictions = svm_r_model.predict(X_test)
rf_predictions = rf_model.predict(X_test)
knn_predictions = knn_model.predict(X_test)
gb_predictions = gb_model.predict(X_test)
ab_predictions = ab_model.predict(X_test)
xgb_predictions = xgb_model.predict(X_test)
gpc_predictions = gpc_model.predict(X_test)


In [31]:
# Combine predictions using ensemble (Voting Classifier)
ensemble_predictions = np.column_stack((cnn_predictions, lstm_predictions, gru_predictions, lr_predictions, 
                                        svm_l_predictions, svm_p_predictions, svm_r_predictions, 
                                        rf_predictions, knn_predictions, gb_predictions, 
                                        ab_predictions, xgb_predictions, gpc_predictions))


In [32]:
# Perform majority voting to get the final prediction
final_predictions = np.mean(ensemble_predictions, axis=1) >= 0.5
final_predictions = final_predictions.astype("int32")

In [33]:
# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.7778


In [34]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, final_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, final_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Confusion Matrix:
[[45  4]
 [14 18]]
Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.92      0.83        49
           1       0.82      0.56      0.67        32

    accuracy                           0.78        81
   macro avg       0.79      0.74      0.75        81
weighted avg       0.78      0.78      0.77        81

Ensemble Accuracy: 0.7778


In [35]:
# Combine predictions using ensemble (Voting Classifier)
ensemble_predictions = np.column_stack((cnn_predictions, lstm_predictions, gru_predictions, svm_r_predictions, 
                                        rf_predictions, knn_predictions, gb_predictions, 
                                        xgb_predictions, gpc_predictions))


In [36]:
# Perform majority voting to get the final prediction
final_predictions = np.mean(ensemble_predictions, axis=1) >= 0.5
final_predictions = final_predictions.astype("int32")

# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, final_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, final_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.7901
Confusion Matrix:
[[46  3]
 [14 18]]
Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.94      0.84        49
           1       0.86      0.56      0.68        32

    accuracy                           0.79        81
   macro avg       0.81      0.75      0.76        81
weighted avg       0.80      0.79      0.78        81

Ensemble Accuracy: 0.7901
